Getting data from google drive and unzipping

In [1]:
# !apt install unzip
# !mkdir birds_sml
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip /content/drive/MyDrive/225Birds.zip -d birds_sml/

**Importing Packages**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16

**Making Directories for training and testing data**

In [2]:
train_dir = 'train/'

test_dir = 'test/'

valid_dir = 'valid/'

img_width, img_height = 224, 224  # Default input size for VGG16

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

**Instantiating the convolutional base**

In [4]:
conv_base = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))

# Show architecture
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

**Extract Features**

In [5]:
import os, shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count,225))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='categorical')
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(train_dir, 31316)  # Agree with our small dataset size
#validation_features, validation_labels = extract_features(valid_dir, 1125)
test_features, test_labels = extract_features(test_dir, 1125)

Found 31316 images belonging to 225 classes.
Found 1125 images belonging to 225 classes.


In [6]:
print(train_features.shape)
test_features.shape

(31316, 7, 7, 512)


(1125, 7, 7, 512)

In [7]:
print(train_labels.shape)
test_labels.shape

(31316, 225)


(1125, 225)

In [8]:
Y_train = np.array([np.where(i == 1)[0][0] for i in train_labels])
Y_test = np.array([np.where(i == 1)[0][0] for i in test_labels])

In [9]:
print(Y_train.shape)
Y_test.shape

(31316,)


(1125,)

In [10]:
print(train_features.shape)
test_features.shape

(31316, 7, 7, 512)


(1125, 7, 7, 512)

In [11]:
X_train = train_features.flatten().reshape(31316 , -1)
X_test = test_features.flatten().reshape(1125 , -1)

In [14]:
print(X_train.shape)
X_test.shape

(31316, 25088)


(1125, 25088)

**SVM Linear Kernel**

In [ ]:
# from sklearn import svm
# clf = svm.LinearSVC(verbose = 1)

# clf.fit(X_train , Y_train)

**SVM Radial Kernel**

In [13]:
from sklearn.svm import SVC

In [14]:
clf = SVC(kernel='rbf' , decision_function_shape='ovr' , verbose=True)
clf.fit(X_train , Y_train)

[LibSVM]

SVC(verbose=True)

In [ ]:
# import pickle
# filename = 'svm_model.sav'
# pickle.dump(clf , open(filename , 'wb'))

In [ ]:
# Y_train.shape
# Y_test.shape

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

In [ ]:
print(accuracy_score(Y_train , y_pred_train))
accuracy_score(Y_test , y_pred_test)

In [ ]:
y_pred